<a href="https://colab.research.google.com/github/andreaco/SpokenDigitClassification/blob/master/SpokenDigitClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spoken Digit Classification

In [0]:
import numpy as np
import re
import os
import librosa
from tqdm.notebook import tqdm

In [0]:
! pip3 install tqdm
! git clone https://github.com/Jakobovski/free-spoken-digit-dataset
#! pip install -r free-spoken-digit-dataset/pip_requirements.txt # probably not needed

In [0]:
# recordings directory
directory = 'free-spoken-digit-dataset/recordings/'
# regexp matching {digitLabel}_{speakerName}_{index}.wav
regexp = re.compile('([0-9])+_([a-z]+)_([0-9]+).wav') 

digitLabels = []
speakerNames = []
indexes = []
audios = []

#progress bar
pbar = tqdm()
pbar.reset(total=len(os.listdir(directory))) 

for f in os.listdir(directory):
  if f.endswith(".wav"):
    digitLabels. append(regexp.search(f).group(1))
    speakerNames.append(regexp.search(f).group(2))
    indexes. append(int(regexp.search(f).group(3)))
    y, sr = librosa.load(directory+f, sr=None)
    audios.append(y)

    pbar.update()
pbar.refresh()  

digitLabels = np.array(digitLabels)
speakerNames = np.array(speakerNames)
indexes = np.array(indexes)
audio = np.array(audio)

print("Elements in train set: {}".format(np.sum(indexes>4)))
print("Elements in test set: {}".format(np.sum(indexes<=4)))

In [0]:
infos_list[:,2]


In [0]:
audio.shape

In [0]:
# different cell
iterable = range(100)
pbar.reset(total=len(iterable))  # initialise with new `total`
for i in iterable:
    pbar.update()
pbar.refresh()  # force print final status but don't `close()`